# CSE 291 Homework3 - SVM  

#### Chao Yu 
#### Wen Liang

## SVM
---
### MNIST
#### Read the data and split to training and test sets

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, zero_one_loss

import warnings

warnings.filterwarnings('ignore')

mnist_train_path = './MNIST/train.csv'
mnist_train_df = pd.read_csv(mnist_train_path, header=None).T
last_column = mnist_train_df.columns[-1]
mnist_train_df.rename(columns={last_column:'Class'}, inplace=True)

In [2]:
mnist_train_df = mnist_train_df.convert_objects(convert_numeric = True)

In [3]:
X = mnist_train_df.drop(0, axis = 0).drop(0, axis = 1)
y = mnist_train_df.drop(0, axis = 1).as_matrix().T[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X.T, y, test_size=0.25, random_state=42)

In [4]:
print y_train

[7 8 5 ..., 2 6 0]


In [5]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

In [6]:
X_train2 = X_train[0:12000]

In [7]:
y_train2 = y_train[0:12000]

In [8]:
clf = ExtraTreesClassifier()
clf = clf.fit(X_train2, y_train2)
clf.feature_importances_  
model = SelectFromModel(clf, prefit=True)
X_train2 = model.transform(X_train2)

In [9]:
X_train2.shape

(12000, 261)

In [10]:
X_test2 = model.transform(X_test)

In [11]:
X_test2.shape

(10500, 261)

In [12]:
from sklearn import svm

#### Test best model

In [13]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 20).fit(X_train2)
X_train3 = pca.transform(X_train2)
X_test3 = pca.transform(X_test2)

rbf_clf = svm.SVC(kernel='rbf', gamma =  1e-06, C =  1000).fit(X_train3, y_train2)
y_validation_pred=rbf_clf.predict(X_test3)   
num_correct = np.sum(y_validation_pred == y_test)
accuracy = float(num_correct) / len(y_test)

print accuracy

0.966285714286


#### linear kernel

In [13]:
num_folds = 5
X_train_folds = []
y_train_folds = []
X_train_folds = np.array(np.array_split(X_train2, num_folds))  #X_train
y_train_folds = np.array(np.array_split(y_train2, num_folds))  #y_train
num_test_sample=len(X_train2)/num_folds

In [28]:
## linear
for c in [0.1, 1, 10, 100, 1000]:
    linear=[]
    for n in range(0,num_folds):
        temp= [x for x in range(0,num_folds) if x != n]
        X_train_dat=np.concatenate(X_train_folds[temp])
        y_train_dat=np.concatenate(y_train_folds[temp])
        lin_clf = svm.SVC(kernel='linear', C=c)
        lin_clf = lin_clf.fit(X_train_dat, y_train_dat)
        y_validation_pred=lin_clf.predict(X_train_folds[n])   
        num_correct = np.sum(y_validation_pred == y_train_folds[n])
        accuracy = float(num_correct) / num_test_sample
        linear.append(accuracy)
    print linear
    print "accuracy (C = :", c, '): ', np.mean(linear)

[0.885, 0.8783333333333333, 0.8858333333333334, 0.8875, 0.8954166666666666]
accuracy (C = : 0.1 ):  0.886416666667
[0.885, 0.8783333333333333, 0.8858333333333334, 0.8875, 0.8954166666666666]
accuracy (C = : 1 ):  0.886416666667
[0.885, 0.8783333333333333, 0.8858333333333334, 0.8875, 0.8954166666666666]
accuracy (C = : 10 ):  0.886416666667
[0.885, 0.8783333333333333, 0.8858333333333334, 0.8875, 0.8954166666666666]
accuracy (C = : 100 ):  0.886416666667
[0.885, 0.8783333333333333, 0.8858333333333334, 0.8875, 0.8954166666666666]
accuracy (C = : 1000 ):  0.886416666667


this dataset is linearly seprable.
The error on training set is zero!
So changing the C won't help.

#### Polynomial kernel

In [31]:
num_folds = 5
X_train_folds = []
y_train_folds = []
X_train_folds = np.array(np.array_split(X_train2, num_folds))  #X_train
y_train_folds = np.array(np.array_split(y_train2, num_folds))  #y_train
num_test_sample=len(X_train2)/num_folds

In [41]:
## poly
for g in [1.0/1000, 1.0/500,1.0/250, 1.0/100, 1.0/50, 1.0, 10.0]:
    for c in [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]:
        print 'gamma = ', g, 'C = ', c
        poly = []
        for n in range(0,num_folds):
            temp= [x for x in range(0,num_folds) if x != n]
            X_train_dat=np.concatenate(X_train_folds[temp])
            y_train_dat=np.concatenate(y_train_folds[temp])
            poly_clf = svm.SVC(kernel='poly', C=c, gamma = g)
            poly_clf = poly_clf.fit(X_train_dat, y_train_dat)
            y_validation_pred=poly_clf.predict(X_train_folds[n])   
            num_correct = np.sum(y_validation_pred == y_train_folds[n])
            accuracy = float(num_correct) / num_test_sample
            poly.append(accuracy)
        print poly
        print "accuracy: ", np.mean(poly)

gamma =  0.001 C =  0.01
[0.9583333333333334, 0.9508333333333333, 0.9570833333333333, 0.95, 0.9554166666666667]
accuracy:  0.954333333333
gamma =  0.001 C =  0.1
[0.9583333333333334, 0.9508333333333333, 0.9570833333333333, 0.95, 0.9554166666666667]
accuracy:  0.954333333333
gamma =  0.001 C =  1.0
[0.9583333333333334, 0.9508333333333333, 0.9570833333333333, 0.95, 0.9554166666666667]
accuracy:  0.954333333333
gamma =  0.001 C =  10.0
[0.9583333333333334, 0.9508333333333333, 0.9570833333333333, 0.95, 0.9554166666666667]
accuracy:  0.954333333333
gamma =  0.001 C =  100.0
[0.9583333333333334, 0.9508333333333333, 0.9570833333333333, 0.95, 0.9554166666666667]
accuracy:  0.954333333333
gamma =  0.001 C =  1000.0
[0.9583333333333334, 0.9508333333333333, 0.9570833333333333, 0.95, 0.9554166666666667]
accuracy:  0.954333333333
gamma =  0.002 C =  0.01
[0.9583333333333334, 0.9508333333333333, 0.9570833333333333, 0.95, 0.9554166666666667]
accuracy:  0.954333333333
gamma =  0.002 C =  0.1
[0.958333

#### Gaussian Kernel

In [16]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
import scipy

In [14]:
## gaussian
for g in [1.0/1000, 1.0/500,1.0/250, 1.0/100]:
    for c in [1.0, 10, 100]:
        print 'gamma = ', g, 'C = ', c
        rbf = []
        for n in range(0,num_folds):
            temp= [x for x in range(0,num_folds) if x != n]
            X_train_dat=np.concatenate(X_train_folds[temp])
            y_train_dat=np.concatenate(y_train_folds[temp])
            rbf_clf = svm.SVC(kernel='rbf', C=c, gamma = g)
            rbf_clf = rbf_clf.fit(X_train_dat, y_train_dat)
            y_validation_pred=rbf_clf.predict(X_train_folds[n])   
            num_correct = np.sum(y_validation_pred == y_train_folds[n])
            accuracy = float(num_correct) / num_test_sample
            rbf.append(accuracy)
        print rbf
        print "accuracy: ", np.mean(rbf)

gamma =  0.001 C =  1.0


KeyboardInterrupt: 

In [21]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 20).fit(X_train2)
X_train3 = pca.transform(X_train2)

In [22]:
num_folds = 5
X_train_foldspca = []
y_train_foldspca = []
X_train_foldspca = np.array(np.array_split(X_train3, num_folds))  #X_train
y_train_foldspca = np.array(np.array_split(y_train2, num_folds))  #y_train
num_test_sample=len(X_train3)/num_folds

In [23]:
## gaussian with PCA
for g in [1e-8, 1e-7, 1e-6, 1e-5]:
    for c in [0.001, 0.01, 1.0, 10, 100, 1000]:
        print 'gamma = ', g, 'C = ', c
        rbf = []
        for n in range(0,num_folds):
            temp= [x for x in range(0,num_folds) if x != n]
            X_train_dat=np.concatenate(X_train_foldspca[temp])
            y_train_dat=np.concatenate(y_train_foldspca[temp])
            rbf_clf = svm.SVC(kernel='rbf', C=c, gamma = g)
            rbf_clf = rbf_clf.fit(X_train_dat, y_train_dat)
            y_validation_pred=rbf_clf.predict(X_train_foldspca[n])   
            num_correct = np.sum(y_validation_pred == y_train_foldspca[n])
            accuracy = float(num_correct) / num_test_sample
            rbf.append(accuracy)
        print rbf
        print "accuracy: ", np.mean(rbf)

gamma =  1e-08 C =  0.001
[0.11125, 0.11125, 0.1175, 0.11333333333333333, 0.11458333333333333]
accuracy:  0.113583333333
gamma =  1e-08 C =  0.01
[0.11125, 0.11125, 0.1175, 0.11333333333333333, 0.11458333333333333]
accuracy:  0.113583333333
gamma =  1e-08 C =  1.0
[0.865, 0.8745833333333334, 0.8725, 0.87125, 0.87375]
accuracy:  0.871416666667
gamma =  1e-08 C =  10
[0.88875, 0.8945833333333333, 0.8966666666666666, 0.8941666666666667, 0.8970833333333333]
accuracy:  0.89425
gamma =  1e-08 C =  100
[0.9091666666666667, 0.9116666666666666, 0.91875, 0.9075, 0.9145833333333333]
accuracy:  0.912333333333
gamma =  1e-08 C =  1000
[0.94, 0.94125, 0.9479166666666666, 0.9420833333333334, 0.94625]
accuracy:  0.9435
gamma =  1e-07 C =  0.001
[0.11125, 0.11125, 0.1175, 0.11333333333333333, 0.11458333333333333]
accuracy:  0.113583333333
gamma =  1e-07 C =  0.01
[0.6954166666666667, 0.695, 0.7095833333333333, 0.705, 0.6995833333333333]
accuracy:  0.700916666667
gamma =  1e-07 C =  1.0
[0.9145833333333

KeyboardInterrupt: 

#### Sigmoid Kernel

In [26]:
## sigmoid
for g in [1e-9, 1e-8, 1e-7, 1e-6]:
    for c in [0.01, 1.0, 10, 100, 1000]:
        print 'gamma = ', g, 'C = ', c
        sig = []
        for n in range(0,num_folds):
            temp= [x for x in range(0,num_folds) if x != n]
            X_train_dat=np.concatenate(X_train_foldspca[temp])
            y_train_dat=np.concatenate(y_train_foldspca[temp])
            sig_clf = svm.SVC(kernel='sigmoid', C=c, gamma = g)
            sig_clf = sig_clf.fit(X_train_dat, y_train_dat)
            y_validation_pred=sig_clf.predict(X_train_foldspca[n])   
            num_correct = np.sum(y_validation_pred == y_train_foldspca[n])
            accuracy = float(num_correct) / num_test_sample
            sig.append(accuracy)
        print sig
        print "accuracy: ", np.mean(sig)

gamma =  1e-09 C =  0.01
[0.11125, 0.11125, 0.1175, 0.11333333333333333, 0.11458333333333333]
accuracy:  0.113583333333
gamma =  1e-09 C =  1.0
[0.6195833333333334, 0.6108333333333333, 0.61625, 0.6258333333333334, 0.6204166666666666]
accuracy:  0.618583333333
gamma =  1e-09 C =  10
[0.8541666666666666, 0.8583333333333333, 0.8558333333333333, 0.86, 0.85875]
accuracy:  0.857416666667
gamma =  1e-09 C =  100
[0.8816666666666667, 0.8858333333333334, 0.88875, 0.8845833333333334, 0.88625]
accuracy:  0.885416666667
gamma =  1e-09 C =  1000
[0.8908333333333334, 0.88625, 0.8925, 0.8925, 0.8954166666666666]
accuracy:  0.8915
gamma =  1e-08 C =  0.01
[0.11125, 0.11125, 0.1175, 0.11333333333333333, 0.11458333333333333]
accuracy:  0.113583333333
gamma =  1e-08 C =  1.0
[0.8541666666666666, 0.8583333333333333, 0.8554166666666667, 0.86, 0.85875]
accuracy:  0.857333333333
gamma =  1e-08 C =  10
[0.8816666666666667, 0.8858333333333334, 0.88875, 0.8845833333333334, 0.88625]
accuracy:  0.885416666667
gam